# Introducing SIREN
**A tool for predicting pullbacks in equity markets**

## Problem statement
XX

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pickle

from datetime import timedelta

# Custom function
import SIREN_func

In [2]:
# Ensuring our notebook remains tidy
import warnings
warnings.filterwarnings('ignore')

## Creation of custom functions
We will import our custom .py module (SIREN_func) which stores the following functions:
* `eda_clean`: Kicking-starting the EDA process
* `derive_yield_curves`: Calculating 10y3m, 10y2y, 5y2y, 30y2y, 30y3m for US and Euro-area regions
* `roll_diff`: Calculate rolling differences for different time horizons
* `roll_pct_chg`: Calculate rolling percentage changes for different time horizons

## Data collection
The bulk of our data has been sourced from Bloomberg via the API on excel. We will write a loop to pull every sheet and store in a dictionary for future use.

In [3]:
# Importing our dataset(s)
xlsx = pd.ExcelFile('../data/spx_fundamentals.xlsx')

# Reading all sheets to a mapsheet_to_df_map = {}
empty_list = {}
for i in xlsx.sheet_names:
    empty_list[i] = pd.read_excel('../data/spx_fundamentals.xlsx', sheet_name=i, skiprows=11, parse_dates=['date'])

In [4]:
# Preview dictionary of dataframes
empty_list.keys()

dict_keys(['econ_sur', 'usd', 'epu', 'finc', 'pe', 'pb', 'eq_indices', 'como', 'credit', 'pct52w', 'vol', 'aaii', 'us_cftc', 'put_call', 'us_yields', 'eu_yields', 'eurdollar', 'fra_ois'])

## Data cleaning, EDA, Feature Engineering
We will once again rely on a combination of loops and custom functions to clean and tidy our dataset(s). We will also create some features for the following datasets:

* No engineering required (values-only; non-stationary)
    - XX
* Rolling differences across different horizons (stationary)
    - Citi economic surprise indices (`econ_sur`)
    - US, Euro-area rates (`us_yields`, `eu_yields`)
        + Yield curves were also calculated
    - Credit spreads (`credit`)
* Rolling **percentage** changes across different horizons (stationary)
    - Equity indices (`eq_indices`)
    - USD indices (`usd`)
    - Commodities (`como`)
    - 12-month forward P/E ratios (`pe`)
    - 12-month forward P/B ratios (`pb`)

### Data Cleaning
**Calculating differences across different horizons**

In [5]:
roll_d = {}
for df in ['econ_sur', 'credit', 'us_yields', 'eu_yields']:
    if df in ['credit']:
        roll_d[f'{df}_chg'] = SIREN_func.roll_diff(SIREN_func.fix_credit(empty_list[df]))
    
    elif df in ['us_yields', 'eu_yields']:
        roll_d[f'{df}_chg'] = SIREN_func.roll_diff(SIREN_func.derive_yield_curves(empty_list[df]))

    else:
        roll_d[f'{df}_chg'] = SIREN_func.roll_diff(empty_list[df])

**Calculating percentage changes across different time horizons**

In [6]:
roll_d2 = {}
for df in ['usd', 'pe', 'pb', 'como', 'eq_indices']:
    if df in ['eq_indices']:
        roll_d2[f'{df}_4w_return'] = SIREN_func.lag_roll_pct_chg(empty_list[df], 4)

    else: 
        roll_d2[f'{df}_chg'] = SIREN_func.roll_pct_chg(empty_list[df]) # Tidy up with custom module later

**Storing the remaining time-series in a dictionary**

In [7]:
d3 = {}
for df in ['epu', 'finc', 'pct52w', 'vol', 'aaii', 'us_cftc', 'put_call']:
    if df in ['us_cftc']:
        d3[df] = SIREN_func.fix_cftc(empty_list[df])
    
    else: d3[df] = SIREN_func.adjust_dates_only(empty_list[df])


In [8]:
final_dict = {**roll_d, **roll_d2, **d3}

In [9]:
final_dict.keys()

dict_keys(['econ_sur_chg', 'credit_chg', 'us_yields_chg', 'eu_yields_chg', 'usd_chg', 'pe_chg', 'pb_chg', 'como_chg', 'eq_indices_4w_return', 'epu', 'finc', 'pct52w', 'vol', 'aaii', 'us_cftc', 'put_call'])

**Tidying earnings revision ratios**

In [10]:
# Read .xlsx highlighting S&P 500's earnings revisions
eri = pd.read_excel("../data/spx_eri.xlsx", sheet_name="Combined", parse_dates=['date'])

In [11]:
# Pipe custom function to calculate ERIs and 4, 13-week differences
eri_chg = SIREN_func.eri_diff(eri, 4, 13)

In [12]:
# Combining all our datasets
full = eri_chg.copy()
for df in final_dict.keys():
    full = pd.merge(left=full, right=final_dict[df], how='left', on='date')

In [13]:
# Previewing our final dataset
SIREN_func.eda_clean(full)

Dataset Statistics:
Shape of dataframe: (862, 285)
% of Null values in dataframe: 1.19%
% duplicate rows: 0.0%

Column names: Index(['eri', 'eri_1m_chg', 'eri_3m_chg', 'cesiusd_1w_chg', 'cesieur_1w_chg',
       'cesigbp_1w_chg', 'cesijpy_1w_chg', 'cesicny_1w_chg', 'cesiglf_1w_chg',
       'cesiusd_4w_chg',
       ...
       'skew', 'aaii_bull', 'aaii_bear', 'aaii_neut', 'cftc_nc_net',
       'cftc_nc_long', 'cftc_nc_short', 'cftc_oi', 'cftc_nc_net_pct_oi',
       'cboe_us'],
      dtype='object', length=285)
Columns Count: 
float64    285
dtype: int64


In [14]:
# Let's drop all missing values
full.dropna(inplace=True)

In [15]:
# Let's save this down as .csv
full.to_pickle('../data/full.pkl')